In [5]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score # <--- ADDED THIS IMPORT
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import GridSearchCV
from sklearn.utils.class_weight import compute_class_weight

# 피처 정의
HE_HP_input_features = ['age','HE_sbp1', 'HE_dbp1', 'HE_wc', 'HE_glu', 'HE_BMI', 'HE_chol', 'HE_LDL_drct', 'HE_HbA1c']
HE_DM_HbA1c_input_features = ['HE_glu','HE_HbA1c', 'HE_wc', 'HE_BMI', 'HE_LDL_drct', 'age'] 
HE_obe_input_features= ['age','HE_wc', 'HE_BMI', 'HE_ht', 'HE_wt', 'HE_alt', 'HE_HDL_st2', 'HE_TG']

# 데이터 로드
#파일 위치 바꿔주세요
data=pd.read_csv("최종_건강검진_데이터셋(가공된).csv")

# HE_HP 데이터 처리
X_hp = data[HE_HP_input_features]
y_hp = data['HE_HP'].astype(int) - 1

# HE_DM_HbA1c 데이터 처리
X_dm = data[HE_DM_HbA1c_input_features]
y_dm = data['HE_DM_HbA1c'].astype(int) - 1

# HE_obe 데이터 처리
X_obe = data[HE_obe_input_features].copy()
y_obe = data['HE_obe'].astype(int) - 1

# 훈련/테스트 분할 (HE_HP)
X_train_hp, X_test_hp, y_train_hp, y_test_hp = train_test_split(X_hp, y_hp, stratify=y_hp, test_size=0.2, random_state=42)

# 훈련/테스트 분할 (HE_DM_HbA1c)
X_train_dm, X_test_dm, y_train_dm, y_test_dm = train_test_split(X_dm, y_dm, stratify=y_dm, test_size=0.2, random_state=42)

# 훈련/테스트 분할 (HE_obe)
X_train_obe, X_test_obe, y_train_obe, y_test_obe = train_test_split(X_obe, y_obe, stratify=y_obe, test_size=0.2, random_state=42)

# 스케일링 (HE_HP)
scaler_hp = StandardScaler()
X_train_hp_scaled = scaler_hp.fit_transform(X_train_hp)
X_test_hp_scaled = scaler_hp.transform(X_test_hp)

# 스케일링 (HE_DM_HbA1c)
scaler_dm = StandardScaler()
X_train_dm_scaled = scaler_dm.fit_transform(X_train_dm)
X_test_dm_scaled = scaler_dm.transform(X_test_dm)

# 스케일링 (HE_obe)
scaler_obe = StandardScaler()
X_train_obe_scaled = scaler_obe.fit_transform(X_train_obe)
X_test_obe_scaled = scaler_obe.transform(X_test_obe)

# SMOTE 적용 (HE_HP)
smote_hp = SMOTE(random_state=42)
X_train_hp_smote, y_train_hp_smote = smote_hp.fit_resample(X_train_hp_scaled, y_train_hp)

# SMOTE 적용 (HE_DM_HbA1c)
smote_dm = SMOTE(random_state=42)
X_train_dm_smote, y_train_dm_smote = smote_dm.fit_resample(X_train_dm_scaled, y_train_dm)

# SMOTE 적용 (HE_obe)
smote_obe = SMOTE(random_state=42)
X_train_obe_smote, y_train_obe_smote = smote_obe.fit_resample(X_train_obe_scaled, y_train_obe)

# 클래스 가중치 계산 (HE_HP)
classes = np.unique(y_train_hp) # 원본 훈련 데이터의 클래스 레이블
weights = compute_class_weight('balanced', classes=classes, y=y_train_hp)
# 계산된 가중치를 딕셔너리 형태로 변환 {클래스_라벨: 가중치}
class_weights_dict_hp = dict(zip(classes, weights))

# 클래스 가중치 계산 (HE_DM_HbA1c)
classes = np.unique(y_train_dm) # 원본 훈련 데이터의 클래스 레이블
weights = compute_class_weight('balanced', classes=classes, y=y_train_dm)
# 계산된 가중치를 딕셔너리 형태로 변환 {클래스_라벨: 가중치}
class_weights_dict_dm = dict(zip(classes, weights))

# 클래스 가중치 계산 (HE_obe)
classes = np.unique(y_train_obe) # 원본 훈련 데이터의 클래스 레이블
weights = compute_class_weight('balanced', classes=classes, y=y_train_obe)
# 계산된 가중치를 딕셔너리 형태로 변환 {클래스_라벨: 가중치}
class_weights_dict_obe = dict(zip(classes, weights))


# 모델 정의 (HistGradientBoostingClassifier)
base_model_hp = HistGradientBoostingClassifier(
    learning_rate=0.1,
    max_iter=100,
    max_depth=5,
    min_samples_leaf=20,
    l2_regularization=1.0,
    random_state=0,
    class_weight=class_weights_dict_hp
)

base_model_dm = HistGradientBoostingClassifier(
    learning_rate=0.1,
    max_iter=100,
    max_depth=5,
    min_samples_leaf=20,
    l2_regularization=1.0,
    random_state=0,
    class_weight=class_weights_dict_dm
)

base_model_obe = HistGradientBoostingClassifier(
    learning_rate=0.05,
    max_iter=50,
    # max_depth=5,
    # min_samples_leaf=20,
    # l2_regularization=1.0,
    random_state=0,
    class_weight=class_weights_dict_obe
)

#모델 훈련
#(HE_HP)
model_hp = CalibratedClassifierCV(base_model_hp, method='sigmoid', cv=5)
model_hp.fit(X_train_hp_smote, y_train_hp_smote)

#(HE_DM_HbA1c)
model_dm = CalibratedClassifierCV(base_model_dm, method='sigmoid', cv=5)
model_dm.fit(X_train_dm_smote, y_train_dm_smote)

#(HE-obe)
model_obe = CalibratedClassifierCV(base_model_obe, method='sigmoid', cv=5)
model_obe.fit(X_train_obe_smote, y_train_obe_smote)
#base_model_obe.fit(X_train_obe_smote, y_train_obe_smote)

#테스트 데이터 평가
#(HE_HP)
y_pred_hp = model_hp.predict(X_test_hp_scaled)
accuracy_hp = accuracy_score(y_test_hp, y_pred_hp) * 100
print("HE_HP Classification Report:")
print(classification_report(y_test_hp, y_pred_hp, target_names=['Normal', 'Pre-hp', 'Pre-high-hp', 'High-hp']))
print(f"HE_HP Accuracy: {accuracy_hp:.1f}%\n") # 정확도 출력
#(HE_DM_HbA1c)
y_pred_dm = model_dm.predict(X_test_dm_scaled)
accuracy_dm = accuracy_score(y_test_dm, y_pred_dm) * 100
print("HE_DM_HbA1c Classification Report:")
print(classification_report(y_test_dm, y_pred_dm, target_names=['Class 0 (Normal)', 'Class 1 (Intermediate)', 'Class 2 (Diabetes)']))
print(f"HE_DM_HbA1c Accuracy: {accuracy_dm:.1f}%\n") # 정확도 출력
#(HE_obe)
y_pred_obe = model_obe.predict(X_test_obe_scaled)
accuracy_obe = accuracy_score(y_test_obe, y_pred_obe) * 100
print("HE_obe Classification Report:")
print(classification_report(y_test_obe, y_pred_obe, target_names=['Low', 'Normal', 'Pre-obese', 'Obese1', 'Obese2', 'Obese3']))
print(f"HE_obe Accuracy: {accuracy_obe:.1f}%\n") # 정확도 출력


# --- Model and Preprocessing Object Saving ---

# 모델 저장 
#(HE_HP)
calibrated_filename_hp = "For_HE_HP_calibrated_real.pkl"
joblib.dump(model_hp, calibrated_filename_hp)
print(f"HE_HP 모델이 '{calibrated_filename_hp}' 파일로 저장되었습니다.")
#(HE_DM_HbA1c)
calibrated_filename_dm = "For_HE_DM_HbA1c_calibrated_real.pkl"
joblib.dump(model_dm, calibrated_filename_dm)
print(f"HE_DM_HbA1c 모델이 '{calibrated_filename_dm}' 파일로 저장되었습니다.")
#(HE_obe)
calibrated_filename_obe = "For_HE_obe_calibrated_real.pkl"
joblib.dump(model_obe, calibrated_filename_obe)
print(f"HE_DM_HbA1c 모델이 '{calibrated_filename_obe}' 파일로 저장되었습니다.")


# 전처리 객체 저장 
#(HE_HP)
scaler_filename_hp = "HE_HP_scaler_real.pkl"
joblib.dump(scaler_hp, scaler_filename_hp)
print(f"HE_HP 스케일러가 '{scaler_filename_hp}' 파일로 저장되었습니다.")
#(HE_DM_HbA1c)
scaler_filename_dm = "HE_DM_HbA1c_scaler_real.pkl"
joblib.dump(scaler_dm, scaler_filename_dm)
print(f"HE_DM_HbA1c 스케일러가 '{scaler_filename_dm}' 파일로 저장되었습니다.")
#(HE_obe)
scaler_filename_obe = "HE_obe_scaler_real.pkl"
joblib.dump(scaler_obe, scaler_filename_obe)
print(f"HE_DM_HbA1c 스케일러가 '{scaler_filename_obe}' 파일로 저장되었습니다.")


# 피처 목록 저장 
#(HE_HP)
features_filename_hp = "HE_HP_input_features_list_real.pkl"
joblib.dump(HE_HP_input_features, features_filename_hp)
print(f"HE_HP 피처 목록이 '{features_filename_hp}' 파일로 저장되었습니다.")
#(HE_DM_HbA1c)
features_filename_dm = "HE_DM_HbA1c_input_features_list_real.pkl"
joblib.dump(HE_DM_HbA1c_input_features, features_filename_dm)
print(f"HE_DM_HbA1c 피처 목록이 '{features_filename_dm}' 파일로 저장되었습니다.")
#(HE_obe)
features_filename_obe = "HE_obe_input_features_list_real.pkl"
joblib.dump(HE_obe_input_features, features_filename_obe)
print(f"HE_DM_HbA1c 피처 목록이 '{features_filename_obe}' 파일로 저장되었습니다.")


# 훈련 데이터 평균값 저장
# (HE_HP)
train_filename_hp = "train_means_hp_real.pkl"
joblib.dump(X_train_hp.mean(), train_filename_hp)
print(f"HE_HP 훈련 데이터 평균값이 '{train_filename_hp}' 파일로 저장되었습니다.")
#(HE_DM_HbA1c)
train_filename_dm = "train_means_dm_real.pkl"
joblib.dump(X_train_dm.mean(), train_filename_dm)
print(f"HE_DM_HbA1c 훈련 데이터 평균값이 '{train_filename_dm}' 파일로 저장되었습니다.")
#(HE_obe)
train_filename_obe = "train_means_obe_real.pkl"
joblib.dump(X_train_obe.mean(), train_filename_obe) # Corrected to X_train_obe.mean()
print(f"HE_obe 훈련 데이터 평균값이 '{train_filename_obe}' 파일로 저장되었습니다.")

# 모델 정확도를 별도 파일로 저장
model_accuracies_to_save = {
    "당뇨병": accuracy_dm,
    "고혈압": accuracy_hp,
    "비만": accuracy_obe
}
joblib.dump(model_accuracies_to_save, 'model_accuracies.pkl')
print(f"모델 정확도가 'model_accuracies.pkl' 파일로 저장되었습니다.")

HE_HP Classification Report:
              precision    recall  f1-score   support

      Normal       0.87      0.84      0.86      1159
      Pre-hp       0.68      0.73      0.71       635
 Pre-high-hp       0.69      0.73      0.71       929
     High-hp       0.66      0.60      0.63       721

    accuracy                           0.74      3444
   macro avg       0.73      0.73      0.73      3444
weighted avg       0.74      0.74      0.74      3444

HE_HP Accuracy: 74.3%

HE_DM_HbA1c Classification Report:
                        precision    recall  f1-score   support

      Class 0 (Normal)       0.99      0.97      0.98      1608
Class 1 (Intermediate)       0.86      0.94      0.90       923
    Class 2 (Diabetes)       0.93      0.88      0.91       913

              accuracy                           0.94      3444
             macro avg       0.93      0.93      0.93      3444
          weighted avg       0.94      0.94      0.94      3444

HE_DM_HbA1c Accuracy: 94.0%

In [2]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
import json

# 사용자 입력 처리 함수
def process_user_input(user_data, features, scaler, train_means):
    user_df = pd.DataFrame([user_data]).reindex(columns=features)
    user_df = user_df.fillna(train_means)  # 누락된 피처를 평균값으로 채움
    user_scaled = scaler.transform(user_df)
    return user_df, user_scaled

# 예측 함수
def predict_disease_probabilities(user_scaled, model, stages, class_explanations):
    probs = model.predict_proba(user_scaled)[0]
    stage_index = np.argmax(probs)
    stage = stages.get(stage_index, "Unknown")
    explanation = class_explanations.get(stage_index, "No explanation available.")
    class_probs_formatted = [f"{probs[i]:.2f}" for i in range(len(probs))]
    return {
        "probabilities": class_probs_formatted,
        "predicted_class": stage_index + 1,
        "class_name": stage,
        "explanation": explanation
    }

# 각 클래스의 설명
diabetes_class_explanations = {
    0: "혈당 수준이 정상입니다. 당뇨병 위험이 낮습니다.",
    1: "혈당이 정상보다 높지만 당뇨병 경계형으로 간주됩니다 생활습관 개선이 필요합니다.",
    2: "당뇨병으로 예측되었습니다. 전문가와 상담이 필요합니다."
}

hypertension_class_explanations = {
    0: "혈압이 정상 범위에 있습니다.",
    1: "혈압이 정상보다 높지만 고혈압 전단계로 간주됩니다. 건강 관리가 필요합니다.",
    2: "고혈압 전단계로, 생활습관 개선과 치료가 필요할 수 있습니다.",
    3: "고혈압으로 예측되었습니다. 전문가의 진단과 치료가 필요합니다."
}

obesity_class_explanations = {
    0: "저체중입니다. 체중 증가가 필요할 수 있습니다.",
    1: "정상 체중입니다. 유지 관리가 필요합니다.",
    2: "과체중입니다. 체중 감량을 고려할 수 있습니다.",
    3: "비만입니다. 체중 감량과 건강 관리가 필요합니다.",
    4: "고도 비만입니다. 전문가의 진단과 관리가 필요합니다.",
    5: "초고도 비만입니다. 전문가의 도움을 받아야 합니다."
}

# 사용자 입력 데이터 (당뇨병은 정상 수치, 고혈압/비만은 기존 수치 유지)
user_data = {
    'HE_glu': 90.0,         # 공복 혈당 (정상으로 변경)
    'HE_HbA1c': None,         # HbA1c 값을 None으로 설정하여 추정되도록 함
    'HE_ht': 170,
    'HE_wt': 63,             # 키 170에 체중 63이면 BMI 약 21.8 → 정상
    'HE_wc': 78,
    'HE_BMI': 18,
    'HE_sbp1': 135,          # 수축기 혈압 121~139 → 주의 혈압 단계
    'HE_dbp1': 85,           # 이완기 혈압 80~89 → 주의 혈압 단계
    'HE_Upro': 3,
    'HE_chol': 200,
    'HE_HDL_st2': 55,
    'HE_TG': 120,
    'HE_LDL_drct': 110,
    'HE_crea': 0.85,
    'HE_ast': 22,
    'HE_alt': 18,
    'age' : 20,
    #'sex' : 1,
    #'ID' : 100
}

def estimate_hba1c_from_glucose(glu):
    return round((glu + 46.7) / 28.7, 1)

if 'HE_HbA1c' not in user_data or user_data['HE_HbA1c'] is None:
    if 'HE_glu' in user_data and user_data['HE_glu'] is not None and user_data['HE_glu'] > 0:
        estimated_hba1c = estimate_hba1c_from_glucose(user_data['HE_glu'])
        user_data['HE_HbA1c'] = estimated_hba1c
        print(f"당화혈색소(HbA1c) 수치가 없어 공복혈당을 기반으로 {estimated_hba1c}%로 추정했습니다.")
    else:
        print("HbA1c와 공복혈당 모두 없어 평균값으로 대체됩니다.")

# 모델, 스케일러, 피처, 평균값 로드
try:
    model_dm = joblib.load('For_HE_DM_HbA1c_calibrated_real.pkl')
    scaler_dm = joblib.load('HE_DM_HbA1c_scaler_real.pkl')
    features_dm = joblib.load('HE_DM_HbA1c_input_features_list_real.pkl')
    train_means_dm = joblib.load('train_means_dm_real.pkl')

    model_hp = joblib.load('For_HE_HP_calibrated_real.pkl')
    scaler_hp = joblib.load('HE_HP_scaler_real.pkl')
    features_hp = joblib.load('HE_HP_input_features_list_real.pkl')
    train_means_hp = joblib.load('train_means_hp_real.pkl')

    model_obesity = joblib.load('For_HE_obe_calibrated_real.pkl')
    scaler_obesity = joblib.load('HE_obe_scaler_real.pkl')
    features_obesity = joblib.load('HE_obe_input_features_list_real.pkl')
    train_means_obesity = joblib.load('train_means_obe_real.pkl')
except FileNotFoundError as e:
    print(f"Error loading model files: {e}. Please ensure all .pkl files are in the correct directory.")
    exit()

# 모델 정확도 로드
try:
    model_scores = joblib.load('model_accuracies.pkl')
    print("모델 정확도 로드 성공.")
except FileNotFoundError:
    print("경고: 'model_accuracies.pkl' 파일을 찾을 수 없습니다. 모델 정확도가 0으로 처리됩니다.")
    model_scores = {
        "당뇨병": 0.0,
        "고혈압": 0.0,
        "비만": 0.0
    }
except Exception as e:
    print(f"모델 정확도 로드 중 오류 발생: {e}")
    model_scores = {
        "당뇨병": 0.0,
        "고혈압": 0.0,
        "비만": 0.0
    }

# 사용자 입력 처리 및 예측
user_df_dm, user_scaled_dm = process_user_input(user_data, features_dm, scaler_dm, train_means_dm)
user_df_hp, user_scaled_hp = process_user_input(user_data, features_hp, scaler_hp, train_means_hp)
user_df_obesity, user_scaled_obesity = process_user_input(user_data, features_obesity, scaler_obesity, train_means_obesity)

# 당뇨병 예측
diabetes_stages = {0: "정상", 1: "경계형 당뇨", 2: "당뇨병"}
result_dm = predict_disease_probabilities(user_scaled_dm, model_dm, diabetes_stages, diabetes_class_explanations)

# 고혈압 예측
hypertension_stages = {0: "정상 혈압", 1: "주의 혈압 단계", 2: "고혈압 전단계", 3: "고혈압"}
result_hp = predict_disease_probabilities(user_scaled_hp, model_hp, hypertension_stages, hypertension_class_explanations)

# 비만도 예측
obesity_stages = {0: "저체중", 1: "정상 체중", 2: "과체중", 3: "비만", 4: "고도비만", 5: "초고도 비만"}
result_obesity = predict_disease_probabilities(user_scaled_obesity, model_obesity, obesity_stages, obesity_class_explanations)

def generate_disease_report(user_name, disease_name, result, top_feature_name, user_value):
    normal_class_index = {
        "당뇨병": 0,
        "고혈압": 0,
        "비만": 1 # Normal is index 1 for obesity
    }

    normal_index = normal_class_index.get(disease_name, 0)
    prob_percent = float(result['probabilities'][normal_index]) * 100
    health_score = round(prob_percent, 1)

    predicted_index = result['predicted_class'] - 1
    predicted_prob = float(result['probabilities'][predicted_index]) * 100

    feature_label_map = {
        "HE_sbp1": "HE_sbp1 (수축기 혈압)",
        "HE_glu": "HE_glu (공복혈당)",
        "HE_BMI": "HE_BMI(BMI)"
    }
    feature_label = feature_label_map.get(top_feature_name, top_feature_name)

    if top_feature_name == "HE_sbp1":
        if user_value <= 120:
            status = "정상"
        elif 121 <= user_value <= 139:
            status = "주의"
        else:
            status = "고위험"
        normal = "90~120"
        warning = "121~139"
    elif top_feature_name == "HE_glu":
        if user_value <= 99:
            status = "정상"
        elif 100 <= user_value <= 125:
            status = "주의"
        else:
            status = "고위험"
        normal = "70~99"
        warning = "100~125"
    elif top_feature_name == "HE_BMI":
        if user_value < 18.5:
            status = "저체중"
        elif 18.5 <= user_value <= 22.9:
            status = "정상"
        elif 23.0 <= user_value <= 24.9:
            status = "과체중"
        elif 25.0 <= user_value <= 29.9:
            status = "비만 1단계"
        elif 30.0 <= user_value <= 34.9:
            status = "비만 2단계"
        else:
            status = "초고도비만"
        normal = "18.5~22.9"
        warning = "23.0~24.9 이상"
    else:
        status = "알 수 없음"
        normal = "-"
        warning = "-"
            
    class_labels_map = {
        "당뇨병": ["정상", "경계형 당뇨", "당뇨병"],
        "고혈압": ["정상 혈압", "주의 혈압 단계", "고혈압 전단계", "고혈압"],
        "비만": ["저체중", "정상 체중", "과체중", "비만", "고도 비만", "초고도 비만"]
    }
    class_labels = class_labels_map.get(disease_name, [f"클래스 {i}" for i in range(len(result["probabilities"]))])

    class_prob_text = ", ".join([
        f"{label} 확률: {float(prob) * 100:.0f}%"
        for label, prob in zip(class_labels, result["probabilities"])
    ])

    current_model_accuracy = model_scores.get(disease_name, None)

    # --- 위험 상태 결정 로직 업데이트 시작 ---
    risk_status_message = ""
    if disease_name == "당뇨병":
        if result['class_name'] == "정상":
            risk_status_message = "정상 상태로 확인됩니다."
        elif result['class_name'] == "경계형 당뇨":
            risk_status_message = "경계형 당뇨 경향이 있습니다."
        elif result['class_name'] == "당뇨병":
            risk_status_message = "당뇨병 위험이 있습니다."
    elif disease_name == "고혈압":
        if result['class_name'] == "정상 혈압":
            risk_status_message = "정상 상태로 확인됩니다."
        elif result['class_name'] == "주의 혈압 단계":
            risk_status_message = "주의 혈압 단계 경향이 있습니다."
        elif result['class_name'] == "고혈압 전단계":
            risk_status_message = "고혈압 전단계 경향이 있습니다."
        elif result['class_name'] == "고혈압":
            risk_status_message = "고혈압 위험이 있습니다."
    elif disease_name == "비만":
        if result['class_name'] == "저체중":
            risk_status_message = "저체중 경향이 있습니다."
        elif result['class_name'] == "정상 체중":
            risk_status_message = "정상 상태로 확인됩니다."
        elif result['class_name'] == "과체중":
            risk_status_message = "과체중 경향이 있습니다."
        elif result['class_name'] in ["비만", "고도비만", "초고도 비만"]:
            risk_status_message = f"{result['class_name']} 위험이 있습니다."
        else:
            risk_status_message = "알 수 없는 비만 관련 상태입니다."
    # --- 위험 상태 결정 로직 업데이트 끝 ---

    # print(f"=== {disease_name} 예측 결과 ===")
    # print(f"1. {user_name}님, 현재 {disease_name} 관련 상태는 {risk_status_message}")
    # print(f"   ({class_prob_text})\n")

    # print(f"2. 건강 점수: {health_score:.1f}점 (이 점수는 정상 확률 기준입니다.)\n")

    # print(f"3. {feature_label} 수치: {user_value} → 상태: {status}")
    # print(f"정상 기준: {normal}, 주의 기준: {warning}")
    # print(f"{disease_name}과 가장 높게 연관된 {feature_label.split()[0]}은(는) 높아지면 {disease_name} 위험이 증가합니다.\n")

    return health_score, risk_status_message

# 대표 feature 맵
top_features = {
    "당뇨병": "HE_glu",
    "고혈압": "HE_sbp1",
    "비만": "HE_BMI"
}

# 질병별 결과를 묶기
disease_results = {
    "당뇨병": result_dm,
    "고혈압": result_hp,
    "비만": result_obesity
}

health_scores = []
disease_risk_statuses = {}

for disease_name, result in disease_results.items():
    top_feature = top_features[disease_name]
    user_value = user_data.get(top_feature, None)

    if user_value is not None:
        score, risk_status = generate_disease_report(
            user_name="홍길동",
            disease_name=disease_name,
            result=result,
            top_feature_name=top_feature,
            user_value=user_value
        )
        health_scores.append(score)
        disease_risk_statuses[disease_name] = risk_status
    else:
        print(f"{disease_name}에 대한 {top_feature} 값이 user_data에 없습니다.")

if health_scores:
    total_score = sum(health_scores) / len(health_scores)

summary_list_of_dictionaries = [] # Change to list of dictionaries for cleaner JSON

feature_name_map = {
    "HE_glu": "공복혈당",
    "HE_sbp1": "수축기 혈압",
    "HE_BMI": "BMI"
}

for disease_name, result in disease_results.items():
    top_feature_code = top_features[disease_name]
    top_feature_name = feature_name_map.get(top_feature_code, top_feature_code)
    user_value = user_data.get(top_feature_code, None)

    # Determine status and normal range based on feature
    status = "알 수 없음"
    normal_range = "-"
    value_unit = ""
    
    if top_feature_code == "HE_sbp1":
        if user_value <= 120:
            status = "정상"
        elif 121 <= user_value <= 139:
            status = "주의"
        else:
            status = "고위험"
        normal_range = "90~120mmHg"
        value_unit = "mmHg"
    elif top_feature_code == "HE_glu":
        if user_value <= 99:
            status = "정상"
        elif 100 <= user_value <= 125:
            status = "주의"
        else:
            status = "고위험"
        normal_range = "70~99mg/dL"
        value_unit = "mg/dL"
    elif top_feature_code == "HE_BMI":
        if user_value < 18.5:
            status = "저체중"
        elif 18.5 <= user_value <= 22.9:
            status = "정상 체중"
        elif 23.0 <= user_value <= 24.9:
            status = "과체중"
        elif 25.0 <= user_value <= 29.9:
            status = "비만"
        elif 30.0 <= user_value <= 34.9:
            status = "고도비만"
        else:
            status = "초고도비만"
        normal_range = "18.5~22.9"
        value_unit = ""

    predicted_class_name = result['class_name']
    
    current_model_accuracy_for_summary = model_scores.get(disease_name, None)
    model_accuracy_str = f"{current_model_accuracy_for_summary:.1f}%" if current_model_accuracy_for_summary is not None else "N/A"

    # Define normal class index for each disease to get the correct "정상 확률"
    normal_class_indices = {
        "당뇨병": 0, # "정상"
        "고혈압": 0, # "정상 혈압"
        "비만": 1   # "정상 체중"
    }
    
    # Get the probability of the normal class
    normal_prob_index = normal_class_indices.get(disease_name, 0) # Default to 0 if not found
    normal_probability = f"{float(result['probabilities'][normal_prob_index]) * 100:.0f}%"

    summary_item = {
        "질병명": disease_name,
        "예측 질병 상태": predicted_class_name
    }

    if predicted_class_name in ["정상", "정상 혈압", "정상 체중"]:
        summary_item["정상 확률"] = normal_probability
        summary_item["멘트"] = "정상 상태로 확인됩니다."
        summary_item["현재 상태 질병 확률"] = "0%" # If normal, the disease probability is 0
    else:
        # For non-normal states, "현재 상태 질병 확률" refers to the probability of the predicted abnormal class
        predicted_prob_of_abnormal_state = f"{float(result['probabilities'][result['predicted_class'] - 1]) * 100:.0f}%"
        summary_item["정상 확률"] = normal_probability # Still show normal probability
        summary_item["현재 상태 질병 확률"] = predicted_prob_of_abnormal_state


    summary_item["모델 정확도(%)"] = model_accuracy_str
    summary_item["지표 정보"] = {
        "지표 이름": top_feature_name,
        "사용자 지표 수치": f"{str(int(user_value)) if user_value == int(user_value) else str(round(user_value, 1))}{value_unit}",
        "정상 지표 기준": f"{normal_range}",
        "사용자 현재 지표 상태": status
    }
    summary_list_of_dictionaries.append(summary_item)

print(json.dumps(summary_list_of_dictionaries, indent=2, ensure_ascii=False))

당화혈색소(HbA1c) 수치가 없어 공복혈당을 기반으로 4.8%로 추정했습니다.
모델 정확도 로드 성공.
[
  {
    "질병명": "당뇨병",
    "예측 질병 상태": "정상",
    "정상 확률": "100%",
    "멘트": "정상 상태로 확인됩니다.",
    "현재 상태 질병 확률": "0%",
    "모델 정확도(%)": "94.3%",
    "지표 정보": {
      "지표 이름": "공복혈당",
      "사용자 지표 수치": "90mg/dL",
      "정상 지표 기준": "70~99mg/dL",
      "사용자 현재 지표 상태": "정상"
    }
  },
  {
    "질병명": "고혈압",
    "예측 질병 상태": "고혈압 전단계",
    "정상 확률": "0%",
    "현재 상태 질병 확률": "95%",
    "모델 정확도(%)": "74.2%",
    "지표 정보": {
      "지표 이름": "수축기 혈압",
      "사용자 지표 수치": "135mmHg",
      "정상 지표 기준": "90~120mmHg",
      "사용자 현재 지표 상태": "주의"
    }
  },
  {
    "질병명": "비만",
    "예측 질병 상태": "저체중",
    "정상 확률": "0%",
    "현재 상태 질병 확률": "100%",
    "모델 정확도(%)": "99.1%",
    "지표 정보": {
      "지표 이름": "BMI",
      "사용자 지표 수치": "18",
      "정상 지표 기준": "18.5~22.9",
      "사용자 현재 지표 상태": "저체중"
    }
  }
]
